In [37]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
from pathos.multiprocessing import ProcessingPool
from pathos.multiprocessing import ProcessPool
import os

#### 1. 크롤링

In [38]:
# 경로 설정

chrome_driver_path = 'G:\내 드라이브\chromedriver_win32\chromedriver.exe'
save_path = 'G:\내 드라이브\LiveCommerse\saveforder_iope'

In [6]:
driver = webdriver.Chrome(chrome_driver_path)
nsl = 'https://shoppinglive.naver.com/channels/64165'
driver.get(nsl)
time.sleep(5)

soup = BeautifulSoup(driver.page_source, 'html.parser')
list_video = soup.select('a.VideoBoxLinkWrapper_wrap_GLkZS')

video_list = []
replay_video_list = []
for i in list_video:
    url = i.get('href')
    video_list.append(url)
    
    if 'replay' in url:
        replay_video_list.append(url)
        
driver.quit()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9664\4198257691.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [8]:
crawlnum1 = replay_video_list[:5]
crawlnum2 = replay_video_list[5:10]

In [31]:
# 크롤러 생성

class MyCrawler:
    def __init__(self, numprocess, webdriverpath):
        self.n_process = numprocess
        self.pool = ProcessPool(nodes=self.n_process)
        self.webdriverpath = webdriverpath
        
    def multi_processing(self, sites, repeat):
        def chat_crawler(url):
            # 사용하는 Library Import
            from selenium import webdriver
            from selenium.webdriver.common.by import By
            import time
            import pandas as pd
            from bs4 import BeautifulSoup

            # 원하는 사이트 로드
            driver = webdriver.Chrome(self.webdriverpath)
            driver.get(url)
            save_number = str(url)
            save_number = save_number[44:50]
            
            # 1분 대기 ... 여러 사이트 열어서 시간이 좀 걸림.
            time.sleep(6*self.n_process)

            # 자동으로 2배속 설정.
            buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
            buttun.click()
            time.sleep(1)
            buttun.click()
            
            # 반복 횟수 설정.
            repeat_n = repeat
            empty = []
            df = pd.DataFrame({'name' : empty, 'chat' : empty})
            
            # 채팅 및 아이디 크롤링.
            for re_n in range(repeat_n):
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
            
                chat_list_temp = []
                for i in temp_list:
                    for j in i:
                        for k in j:
                            chat = k.select('span')
                            chat = str(chat)
                            chat_list_temp.append(chat)

                name_list_temp = []
                for i in temp_list:
                    for j in i:
                        name = j.select('strong')
                        name = str(name)
                        if 'nickname' in name:
                            name_list_temp.append(name)

                chat_list = []
                for i in chat_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        chat_list.append(i)
                    except:
                        continue
                    
                name_list = []
                for i in name_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        name_list.append(i)
                    except:
                        continue
                df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
                df = pd.concat([df, df_temp], ignore_index=True)
                
                time.sleep(150)

            # df 형성 및 저장.
            df.drop_duplicates(inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.to_csv(save_path + '\{}.csv' .format(save_number), index=False)
            
            driver.quit()
        
        self.pool.map(chat_crawler, sites)

In [32]:
# 크롤러 할당

num1 = MyCrawler(len(crawlnum1), chrome_driver_path)
num2 = MyCrawler(len(crawlnum2), chrome_driver_path)

In [33]:
# 크롤러 병렬 실행

num1.multi_processing(crawlnum1, 12)
num2.multi_processing(crawlnum2, 12)

In [76]:
# csv 파일 전체 통합

empty = []
df = pd.DataFrame({'name' : empty, 'chat' : empty})

data_csv = os.listdir(save_path)

for i in data_csv:
    df_temp = pd.read_csv(save_path + f'\{i}')
    df = pd.concat([df, df_temp], ignore_index=True)
    
df.reset_index(drop=True, inplace=True)
df.to_csv(save_path + '\\total.csv', index=False)


In [87]:
df = pd.read_csv(r'G:\내 드라이브\LiveCommerse\saveforder_iope\total.csv')
df.name.nunique()

4520

In [83]:
# 전체 df 가져와서 구매번호 인증 관련된 채팅 제거.

df = pd.read_csv(r'G:\내 드라이브\LiveCommerse\saveforder_iope\total.csv')

pattern = '\d{16,}'
drop_df = df[~(df['chat'].str.contains(pattern))]
drop_df.reset_index(drop=True, inplace=True)
drop_df

,name,chat
0,김따뜻,기대기대🤗
1,져니,안녕하세요
2,eundoki2004,안녕하세요~ㅎㅎ
3,져니,반가워요
4,져니,잘볼게요
...,...,...
10513,wls,당첨되고 시프요
10514,딸기토끼,대박적혜택! 아까 더살껄...
10515,인문논술독서논술과외,헐50종
10516,젤리주주,카페인샷 사은품으로 가는 포텐셜크림이랑 꼭 함께 바르세요. 시너지 폭발이예요!!


In [88]:
drop_df.name.nunique()

4183

##### 아래는 연습장임

In [ ]:
# 크롤러 만들기.
def scrape(url):
    from selenium import webdriver
    import time
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
    driver.get(url)

In [21]:
# 자동 2배속

def sppedx2(driver):
    buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
    buttun.click()
    time.sleep(1)
    buttun.click()

In [22]:
# 자동 크롤링 반복 후 df 형성

def crawler(repeat, savepath):
    repeat_n = repeat
    empty = []
    df = pd.DataFrame({'name' : empty, 'chat' : empty})
    
    for re_n in range(repeat_n):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
        chat_list_temp = []
        for i in temp_list:
            for j in i:
                for k in j:
                    chat = k.select('span')
                    chat = str(chat)
                    chat_list_temp.append(chat)

        name_list_temp = []
        for i in temp_list:
            for j in i:
                name = j.select('strong')
                name = str(name)
                if 'nickname' in name:
                    name_list_temp.append(name)

        chat_list = []
        for i in chat_list_temp:
            try:
                i = (i.split('<')[1])
                i = (i.split('>')[1])
                chat_list.append(i)
            except:
                continue
            
        name_list = []
        for i in name_list_temp:
            try:
                i = (i.split('<')[1])
                i = (i.split('>')[1])
                name_list.append(i)
            except:
                continue
        df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
        df = pd.concat([df, df_temp], ignore_index=True)
        
        time.sleep(150)

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(savepath)

In [20]:
driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
driver.get(replay_video_list[0])

time.sleep(5)
time.sleep(10)

# 자동 2배속
buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
buttun.click()
time.sleep(1)
buttun.click()


repeat = 12

empty = []
df = pd.DataFrame({'name': empty, 'chat' : empty})

for re_n in range(repeat):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
    chat_list_temp = []
    for i in temp_list:
        for j in i:
            for k in j:
                chat = k.select('span')
                chat = str(chat)
                chat_list_temp.append(chat)

    name_list_temp = []
    for i in temp_list:
        for j in i:
            name = j.select('strong')
            name = str(name)
            if 'nickname' in name:
                name_list_temp.append(name)

    chat_list = []
    for i in chat_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            chat_list.append(i)
        except:
            continue
        
    name_list = []
    for i in name_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            name_list.append(i)
        except:
            continue
    
    df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
    df = pd.concat([df, df_temp], ignore_index=True)
    
    print(re_n)
    time.sleep(150)

df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/df_save_{}.csv' .format(url_num))

C:\Users\gkfua\AppData\Local\Temp\ipykernel_7904\3999771474.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')


In [27]:
def scrape(url):
    from selenium import webdriver
    import time
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    
    time.sleep(5)
    time.sleep(10)

    # 자동 2배속
    buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
    buttun.click()
    time.sleep(1)
    buttun.click()
    
    time.sleep(10)

In [28]:
pool = ProcessPool(nodes=10)
pool.map(scrape, replay_video_list)

[None, None, None, None, None, None, None, None, None, None]

In [51]:
class WebCrawler:
    def __init__(self):
        self.pool = ProcessPool(nodes=10)
        
    def open_browser(url):
        from selenium import webdriver
        driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
        driver.get(url)
        
    def multi_processing(self, url_list):
        self.pool.map(open_browser, url_list)


In [18]:
len(replay_video_list)

10

In [17]:
replay_video_list

['https://view.shoppinglive.naver.com/replays/869256?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/820001?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/809363?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/768444?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/719762?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/687769?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/646838?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/621921?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/620591?fm=shoppinglive&sn=home',
 'https://view.shoppinglive.naver.com/replays/595816?fm=shoppinglive&sn=home']

In [6]:
# 원하는 사이트 오픈

url_num = 2
driver = webdriver.Chrome(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/chromedriver_mac64/chromedriver')
driver.get(url_list[url_num])

C:\Users\gkfua\AppData\Local\Temp\ipykernel_19496\3999835344.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/chromedriver_mac64/chromedriver')


NameError: name 'url_list' is not defined

In [75]:
repeat = 12

empty = []
df = pd.DataFrame({'name': empty, 'chat' : empty})

for re_n in range(repeat):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
    chat_list_temp = []
    for i in temp_list:
        for j in i:
            for k in j:
                chat = k.select('span')
                chat = str(chat)
                chat_list_temp.append(chat)

    name_list_temp = []
    for i in temp_list:
        for j in i:
            name = j.select('strong')
            name = str(name)
            if 'nickname' in name:
                name_list_temp.append(name)

    chat_list = []
    for i in chat_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            chat_list.append(i)
        except:
            continue
        
    name_list = []
    for i in name_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            name_list.append(i)
        except:
            continue
    
    df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
    df = pd.concat([df, df_temp], ignore_index=True)
    
    print(re_n)
    time.sleep(150)

df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/df_save_{}.csv' .format(url_num))

0
1
2
3
4
5
6
7
8
9
10
11
